
# <b><font face="微软雅黑" size="5" color="lightblue">Transformer网络详解</font></b>

## <b><font face="微软雅黑" size="4" color="lightblue">导入相关库</font></b>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

## <b><font face="微软雅黑" size="4" color="lightblue">Transformer - Encoder</font></b>

### <font face="微软雅黑" size="3">Input Embedding：首先通过Tokenizer将文本拆分成token，再将token映射为离散的token ID，最终通过词嵌入矩阵（输入权重矩阵，可以随机初始化跟随网络一起训练得到到，也可以设置预训练好的参数）映射到低维稠密的连续空间。</font>

### <font face="微软雅黑" size="3">Positional Encoding（正余弦位置编码）:</font>

<img src="Positional.png" width="697" height="183" alt="正余弦位置编码">

上式中位置pos即指输入token序列中单个token的位置，嵌入向量维度i即指单个token的第i维特征，如果你的词向量维度为512，那么针对这个token的位置编码也会有512维，其中奇数位用上式中的cos公式计算，偶数位用sin公式计算，最后将生成的位置编码与词向量相加，即最后输入enconder的特征。

计算公式中的分母部分称为频率衰减因子：$d_{model}$是词向量的维度，当i越小，分母越小，$\omega = 1/10000^{2i/d_{model}}$会越大，正余弦波形的震荡就会越快，即高频，那么即使位置相邻的token，模型也能清晰感知到编码的变化，反之如果i越大。分母越大，$\omega = 1/10000^{2i/d_{model}}$会越小，正余弦波形震荡缓慢，即低频，那么只有在位置相距较远时，位置编码信息才会有明显的区别；通过这个操作，可以使低维特征更关注精细的位置区分，而高维特征则更注重全局的位置相关性。

正余弦位置编码，本质上是一种通过正弦余弦函数人为构造的、具有唯一性和数学规律性的位置标识。它本身不具备物理意义，而是通过将这种标识与词向量相加，为模型提供了区分序列顺序的信号，从而让缺乏时序感知能力的 Transformer 能够理解文本的语序信息。

In [ ]:
#正余弦位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout = 0.1, max_len=5000):
        """
        为输入序列中的每个token生成位置编码

        参数:
            d_model: 词向量维度
            dropout: dropout概率
            max_len: 最大序列长度
        """
        super.__init__()
        self.dropout = nn.Dropout(p=dropout)
       
        # 创建一个位置编码矩阵
        pe = torch.zeros(max_len, d_model)
        #生成从 0 -（max_len-1） 的位置索引, 并将其维度扩展为 (max_len, 1)
        position = torch.arange(0, max_len).unsqueeze(1)
        
        # 计算每个维度的频率(指数变换后的公式) 维度：(1, d_model/2)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        # 计算正余弦位置编码
        pe[:,0::2] = torch.sin(position * div_term)  # 偶数维度使用正弦函数
        pe[:,1::2] = torch.cos(position * div_term)  # 奇数维度使用余弦函数

        # 增加一个维度，方便后续与输入相加，形状变为 (1, max_len, d_model)
        pe = pe.unsqueeze(0)

        # 将位置编码注册为模型的缓冲区，使其不会被当作参数更新
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 取出与输入序列长度相同的位置编码，将位置编码加到输入上
        X = x + self.pe[:, :x.size(1), :]  
        return self.dropout(X)  # 应用dropout并返回结果

### <font face="微软雅黑" size="3">Encoder输入：文本的token通过Word Embedding映射为固定维度的词向量，一般为512维，再将词向量序列和位置编码相加变为多头注意力网络的输入。</font>

<img src="Encoder.png" width="480" height="400" alt="Encoder">

### <font face="微软雅黑" size="3">单头注意力计算：输入的数据维度为 batch_size, head_nums, seq_length, 词向量维度/head_nums;输入数据分别乘wq，wk，wv 得到Q,K,V，Q,K矩阵点乘输出不同位置词向量和其余位置词向量之间的余弦相似度，再通过Scaling（结果除根号dk,dk是单个头输入的词向量维度）操作防止乘积过大而导致softmax归一化后梯度极小的问题，最后将softmaxs输出的结果和V矩阵相乘。</font>

In [ ]:
#定义参数
embed_size = 512  #词嵌入维度, 每个token经过Embeding后的维度
head_nums = 8     #多头注意力头数
head_size = embed_size // head_nums #每个头的输入维度
hidden_size = 512 #每个头的输出维度

#单个注意力代码实现
#*args：接收任意个数的位置参数，打包成元组 (tuple) 传入函数内部；
#**kwargs：接收任意个数的关键字参数，打包成字典 (dict) 传入函数内部；
class SingleAttention(nn.Module):
    #初始化
    def __init__(self, *args, **kwargs):
        #初始化父类
        super.__init__(*args,**kwargs)
        #定义Q,K,V的线性变换层
        self.Q_linear = nn.Linear(head_size,hidden_size)
        self.K_linear = nn.Linear(head_size,hidden_size)
        self.V_linear = nn.Linear(head_size,hidden_size)

    def forward(self,x):
        batch_size, seq_len, head_size = x.size()
        #获取Q,K,V
        Q = self.Q_linear(x)   #查询矩阵维度：(batch_size, seq_len, hidden_size)
        K = self.K_linear(x)   #键矩阵维度：(batch_size, seq_len, hidden_size) 
        V = self.V_linear(x)   #值矩阵维度：(batch_size, seq_len, hidden_size)

        #计算注意力分数
        #注意力分数维度：(batch_size, seq_len, seq_len) 每个位置与其他位置的相关性
        scores = Q @ K.transpose(-2, -1) / math.sqrt(head_size)  

        #计算注意力权重, 在最后一维上进行softmax归一化，得到每个位置对当前token的权重，且所有权重和为1
        attention_weights = F.softmax(scores, dim=-1)

        #针对每一个token，计算所有位置的加权和
        output = attention_weights @ V
        return output, attention_weights

### <font face="微软雅黑" size="3">Add操作：残差操作，由于在输出中加上了输入x（$F(x) + x$），其梯度为：
$$
\frac{\partial \mathcal{L}}{\partial x} = \frac{\partial \mathcal{L}}{\partial (x + F(x))} \cdot (1 + \frac{\partial F(x)}{\partial x})
$$
如上梯度的计算结果中包含了一个常数项 1，这意味着即使 $\frac{\partial F(x)}{\partial x}$ 很小，梯度仍然可以有效地反向传播，缓解梯度消失问题。</font>

### <font face="微软雅黑" size="3">LayerNorm操作：对每个 token 的 d_model 维向量，做以下三步计算：
（1）减去均值 → 让数据中心在 0;<br>
（2）除以标准差 → 让数据方差为 1;<br>
（3）乘以可学习的缩放因子 γ，加上可学习的偏移因子 β → 让模型恢复表达能力（不被归一化限制死）</font>。

<font face="微软雅黑" size="3" color="lightgreen">层归一化和批归一化的区别</font>： LayerNorm：针对单一batch，单一token的所有特征维度进行归一化;     BatchNorm：针对所有batch的，所有token的同一个特征维度进行归一化。

<font face="微软雅黑" size="3" color="lightgreen">归一化的作用</font>:
1. 解决梯度消失 / 爆炸，让深层网络能训练
梯度消失：如果输入数据分布很散（方差很大），经过激活函数（如 Sigmoid、Tanh）后，梯度会变得极小，参数几乎不更新。
梯度爆炸：如果输入数据分布很极端，梯度会变得极大，参数更新步长失控，模型直接训崩。
作用：把输入数据缩放到均值 0、方差 1的标准范围，让激活函数工作在梯度最敏感的线性区域，保证梯度稳定回传，深层网络也能顺利训练。
2. 加速训练收敛，减少训练时间
没有归一化时，模型需要花大量时间去适应每层变化的分布，收敛极慢。归一化后，每层输入分布稳定，参数更新方向更明确，收敛速度能提升几倍甚至几十倍。
3. 降低对初始化和学习率的敏感程度
没有归一化：参数初始化稍微差一点，或者学习率设大一点，模型就不收敛或直接发散。
有归一化：对初始化不挑剔，学习率可以设得更大，训练更鲁棒。
4. 起到一定的正则化作用，防止过拟合
归一化会在每一层引入噪声（比如 Batch Norm 的批次统计、Layer Norm 的样本统计），这种噪声能抑制模型对训练数据的过度拟合，提升泛化能力。


In [ ]:
embed_size = 512
head_nums = 12
head_size = embed_size // head_nums
hidden_size = 512
dropout = 0.1

# 单个注意力
class Attention(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 计算Q，K，V
        self.query = nn.Linear(head_size, hidden_size)
        self.key = nn.Linear(head_size, hidden_size)
        self.value = nn.Linear(head_size, hidden_size)

        # 创建mask
        self.register_buffer("attention_mask", torch.tril(torch.ones(head_size, head_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_len, head_size = x.size()
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        weight = q @ k.transpose(-2,-1)
        weight = weight.masked_fill(self.attention_mask[:seq_len,:seq_len] == 0, float('-inf')) / math.sqrt(head_size)
        weight = F.softmax(weight, -1)
        weight = self.dropout(weight)
        output = weight @ v
        return  output

多头注意力计算：多头注意力即多个单头的叠加，针对一个词向量来说，假如一个词向量的维度是512，头数为8，那么每个头的输入就是64维，每个头会将词向量的不同部分作为输入计算，关注词向量不同维度下的特征，最后将所有头的输出拼接。多头注意力输出后会经过残差和层归一化操作。

<b><font face="微软雅黑" size="4" color="lightblue">Transformer - Decoder</font></b>